In [2]:
import requests
from bs4 import BeautifulSoup
import unicodedata
import re


/Users/anper/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
wiki_main_page_link = 'https://ru.wiktionary.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%9C%D0%B0%D1%82%D0%B5%D1%80%D0%BD%D1%8B%D0%B5_%D0%B2%D1%8B%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F/ru'

main_page = requests.get(wiki_main_page_link)
main_page

<Response [200]>

In [ ]:
soup = BeautifulSoup(main_page.content, 'html.parser')
soup

In [16]:
title = soup.find('ul')
print(title)

<ul class="vector-menu-content-list">
<li class="mw-list-item" id="n-Заглавная-страница"><a href="/wiki/%D0%92%D0%B8%D0%BA%D0%B8%D1%81%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C:%D0%97%D0%B0%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D0%B0%D1%8F_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0"><span>Заглавная страница</span></a></li><li class="mw-list-item" id="n-Полный-индекс"><a href="/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:%D0%92%D1%81%D0%B5_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D1%8B"><span>Полный индекс</span></a></li><li class="mw-list-item" id="n-Категории"><a href="/wiki/%D0%92%D0%B8%D0%BA%D0%B8%D1%81%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C:%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B8"><span>Категории</span></a></li><li class="mw-list-item" id="n-newpages"><a href="/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:%D0%9D%D0%BE%D0%B2%D1%8B%D0%B5_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D1%8B"><span>Новые страницы</span></a></li><li class="mw-list-item" 

In [ ]:
soup.select('ul li a')

In [4]:
import requests
from bs4 import BeautifulSoup
import re

def fetch_data(page_url, expression):
    try:
        response = requests.get(page_url)
        response.raise_for_status()  # Проверяем статус ответа
        soup = BeautifulSoup(response.content, 'html.parser')
    except Exception as e:
        print(f"Ошибка при запросе или парсинге страницы {page_url}: {e}")
        return {
            "выражение": expression,
            "значение": None,
            "пример_употребления": None,
            "синонимы": None
        }
    
    # Инициализируем переменные значениями по умолчанию
    examples, synonyms, meanings = None, None, None
    
    # Извлекаем пример употребления
    try:
        example_blocks = soup.select('span.example-fullblock')
        examples = [block.get_text(" ", strip=True) for block in example_blocks]
    except Exception as e:
        print(f"Ошибка при извлечении примера употребления: {e}")

    # Извлекаем синонимы
    try:
        synonyms_heading = soup.find("h4", id="Синонимы")
        synonyms_list = synonyms_heading.find_next("ol") if synonyms_heading else None
        synonyms = [a.get_text() for a in synonyms_list.find_all("a")] if synonyms_list else []
    except Exception as e:
        print(f"Ошибка при извлечении синонимов: {e}")

    # Извлекаем значение
    try:
        meaning_section = soup.find("h4", id="Значение")
        meaning_list = meaning_section.find_next("ol") if meaning_section else None
        meanings = []
        
        if meaning_list:
            for li in meaning_list.find_all("li"):
                text = li.get_text(strip=True)
                if text:
                    # Добавляем пробелы после запятых и точек с запятой
                    formatted_text = re.sub(r"([,;])", r"\1 ", text)
                    # Удаляем всё после символа ◆
                    formatted_text = re.split(r"◆", formatted_text)[0].strip()
                    meanings.append(formatted_text)
    except Exception as e:
        print(f"Ошибка при извлечении значения: {e}")
    
    # Структура данных
    data = {
        "выражение": expression,
        "значение": meanings if meanings else None,
        "пример_употребления": examples[0] if examples else None,  # Первый пример
        "синонимы": synonyms if synonyms else None
    }
    
    return data


In [118]:
import requests
from bs4 import BeautifulSoup

# Ссылка на основную страницу
wiki_main = 'https://ru.wiktionary.org'
wiki_main_page_link = '/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%9C%D0%B0%D1%82%D0%B5%D1%80%D0%BD%D1%8B%D0%B5_%D0%B2%D1%8B%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F/ru'
next_page = wiki_main + wiki_main_page_link

# page = requests.get(test)
# soup = BeautifulSoup(page.content, 'html.parser')
# links = []
# for a in soup.select('ul li a'):
#     if ['href', 'title'] == list(a.attrs.keys()):
        
#         pairs = (a.attrs['href'], a.attrs['title'])
#         links.append(pairs)

# words_links = links[6:]
# link, name = words_links[1:2][0]
# link, name

result = []
while next_page:
    # Запрос и парсинг страницы
    page = requests.get(next_page)
    soup = BeautifulSoup(page.content, 'html.parser')
    links = []
    for a in soup.select('ul li a'):
        if ['href', 'title'] == list(a.attrs.keys()):
            
            pairs = (a.attrs['href'], a.attrs['title'])
            links.append(pairs)

    words_links = links
    for n, pair in enumerate(words_links[6:], start=1):
        link, word = pair
        link = wiki_main + link
        print(word, n, len(words_links))
        get_data = fetch_data(link, word)
        result.append(get_data)
    

    # Поиск ссылки на следующую страницу
    next_page_link = soup.find('a', text='Следующая страница')

    # Проверка и формирование полной ссылки
    if next_page_link and 'href' in next_page_link.attrs:
        next_page = wiki_main + next_page_link['href']
        print("Ссылка на следующую страницу:", next_page)
    else:
        print("Ссылка на следующую страницу не найдена.")
        break


а поебаться тебе не завернуть? 1 210
а сто хуёв — большая куча? 2 210
апчхуй 3 210
архипиздрит 4 210
ахуй 5 210
ахулиард 6 210
беден как Адам, и хуя прикрыть нечем 7 210
без пизды 8 210
бля 9 210
блядво 10 210
блядёшка 11 210
блядина 12 210
блядища 13 210
блядки 14 210
блядовать 15 210
блядовитый 16 210
блядоёбина 17 210
блядский 18 210
блядство 19 210
блядствовать 20 210
блядун 21 210
блядь 22 210
блядькать 23 210
блядькнуть 24 210
бляха-муха 25 210
босый хуй 26 210
босым хуем около голой пизды шутки не шутят 27 210
буква «ю» — по хую 28 210
был бы хуй, а подхуйки найдутся 29 210
в ахуе 30 210
в два хуя 31 210
в душе не ебу 32 210
в еблет наклацать 33 210
в жопу ёбаный 34 210
в пизду 35 210
в рай въехать на чужом хую 36 210
в рот ебаться 37 210
в хуй дует 38 210
в хуй дышит 39 210
в хуй не всраться 40 210
в хуй не дуть 41 210
в хуй не ставить 42 210
в хуй не упереться 43 210
веселоебучесть 44 210
веселоебучий 45 210
весь в поте и хуй в роте 46 210
весь в поту и хуй во рту 47 210
вешат

/var/folders/q3/z6chdjmx0_n7h3b2_bcvghs00000gn/T/ipykernel_12248/1545088822.py:44: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  next_page_link = soup.find('a', text='Следующая страница')


Ссылка на следующую страницу: https://ru.wiktionary.org/w/index.php?title=%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%9C%D0%B0%D1%82%D0%B5%D1%80%D0%BD%D1%8B%D0%B5_%D0%B2%D1%8B%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F/ru&pagefrom=%D0%B5%D0%B1%D0%B0%D0%BD%D1%8C%D0%B5%0A%D0%B5%D0%B1%D0%B0%D0%BD%D1%8C%D0%B5#mw-pages
ебанье 1 210
ебанько 2 210
ебарёк 3 210
ебарёчек 4 210
ебаришка 5 210
ебаришко 6 210
ёбарь 7 210
ёбарь-перехватчик 8 210
ебати 9 210
ебатись 10 210
ебаторий 11 210
ебатура 12 210
ебать 13 210
ебать вола 14 210
ебать и резать 15 210
ебать мозги 16 210
ебать мой лысый череп 17 210
ебать мой хуй ментовской жопой 18 210
ебать раком 19 210
ебаться 20 210
ебаться в телевизор 21 210
ебач 22 210
ебашить 23 210
ебашиться 24 210
ебейший 25 210
ебель 26 210
ебельник 27 210
ебёна мать 28 210
ебёна темя 29 210
ебенить 30 210
ебеный 31 210
ебёный 32 210
ебеня 33 210
еби его мать 34 210
еби её мать 35 210
ебись о плетень 36 210
ебись оно всё конём 37 210
ебись твою в ноздрю 38

In [120]:
import pickle

with open('data.pickle', 'wb') as handle:
    pickle.dump(result, handle)

with open('data.pickle', 'rb') as handle:
    result = pickle.load(handle)

In [121]:
result

[{'выражение': 'а поебаться тебе не завернуть?',
  'значение': ['обсц.отрицательныйответна чью-либо просьбу'],
  'пример_употребления': '◆ —\xa0Слышь, Санёк, одолжи пятихатку! — А поебаться тебе не завернуть? Ты мне ещё с прошлого раза штуковину торчишь!',
  'синонимы': ['нет',
   'никогда',
   'ни за что',
   'ни в коем случае',
   'ни при каких условиях',
   'разг.',
   'нетушки',
   'шиш',
   'фиг',
   'фигу',
   'фигу с дрыгой',
   'фигушки',
   'держи карман шире',
   'ни за какие коврижки',
   'жарг.',
   'мол.',
   'а банана не хочешь?',
   'а тайгу тебе не пропылесосить?',
   'а спину тебе вареньем не намазать?',
   'а спинку Вам крыжовенным вареньем не намазать?',
   'а в рот тебе не плюнуть жёваной морковкой?',
   'а шнурки тебе не погладить?',
   'а в носе не засвистит?',
   'разг.',
   'сниж.',
   'в рот тебе ноги у дороги!',
   'эвф.',
   'хрена',
   'хренушки',
   'хрен тебе',
   'хрен тебе в зубы',
   'хрен тебе в грызло',
   'хрен тебе на лысый череп',
   'хера',
   'хе

In [153]:
[i['выражение'] for i in result]

['а поебаться тебе не завернуть?',
 'а сто хуёв — большая куча?',
 'апчхуй',
 'архипиздрит',
 'ахуй',
 'ахулиард',
 'беден как Адам, и хуя прикрыть нечем',
 'без пизды',
 'бля',
 'блядво']

In [162]:
result[-2]

{'выражение': 'бля',
 'значение': ['бран.то же,  чтоблядь;  употребляется для выражения раздражения,  удивления и\xa0т.\xa0п.',
  'частица, вульг.употребляется дляусилениявыразительности,  подчёркивания значения того или иного слова'],
 'пример_употребления': '◆ Ах, что за славная земля Вокруг залива Коктебля; Колхозы, бля , совхозы, бля , Природа! В.\xa0П.\xa0Аксёнов , «Таинственная страсть», 2007 г. [ НКРЯ ]',
 'синонимы': ['блядь', 'ёбть', 'ну']}

In [158]:
response = requests.get('https://ru.wiktionary.org/wiki/%D0%B1%D0%BB%D1%8F%D0%B4%D0%B2%D0%BE')
soup = BeautifulSoup(response.content, 'html.parser')

# Находим раздел "Значение"
# Находим раздел "Значение"
meaning_section = soup.find("h4", id="Значение")

# Получаем следующий элемент за разделом "Значение"
meaning_list = meaning_section.find_next("ol")

# Извлекаем текст всех пунктов значения, добавляем пробелы и обрезаем текст после ◆
meanings = []
for li in meaning_list.find_all("li"):
    text = li.get_text(strip=True)
    if text:
        # Добавляем пробелы после запятых и точек с запятой
        formatted_text = re.sub(r"([,;])", r"\1 ", text)
        # Удаляем всё после символа ◆
        formatted_text = re.split(r"◆", formatted_text)[0].strip()
        meanings.append(formatted_text)

meanings

['собир., крим. жарг.бляди; сообществоблядей']

In [306]:
import pandas as pd

data = result
# Преобразуем значения и синонимы в строки для удобства
for entry in data:
    if 'значение' in entry and isinstance(entry['значение'], list):
        entry['значение'] = "; ".join(entry['значение'])
    if 'синонимы' in entry and isinstance(entry['синонимы'], list):
        entry['синонимы'] = ", ".join(entry['синонимы'])

# Создаем DataFrame
df = pd.DataFrame(data)

df


,выражение,значение,пример_употребления,синонимы
0,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"◆ — Слышь, Санёк, одолжи пятихатку! — А поебат...","нет, никогда, ни за что, ни в коем случае, ни ..."
1,а сто хуёв — большая куча?,"обсц., шутл.используется в качестве ответного ...",◆ Отсутствует пример употребления (см. рекомен...,None
2,апчхуй,"обсц., шутл.то же, чтоапчхи; передаёт звукчи...",◆ Отсутствует пример употребления (см. рекомен...,апчхи
3,архипиздрит,"обсц.бабник, ловелас, дамский угодник, гуляка","◆ Да, Андрюха тот ещё архипиздрит — каждый ден...","разг., бабник, завзятый бабник, беспутник, бес..."
4,ахуй,обсц.сильноеудивлениеилинедоумение; обсц.состо...,"◆ Я с превеликим ахуем принимаю к сведению, чт...","удивление, недоумение, эвф., ахер, обсц., охуе..."
...,...,...,...,...
1170,Участник:DonRumata/Песочница/Стоп-лист,None,None,None
1171,Категория:Матерные выражения по языкам,None,None,None
1172,Категория:Русские выражения по стилистической ...,None,None,None
1173,Категория:Табуированная лексика/ru,None,None,None


In [18]:
df['выражение'].iloc[0], df['значение'].iloc[0], df['пример_употребления'].iloc[0], df['синонимы'].iloc[0]

('а поебаться тебе не завернуть?',
 'обсц.отрицательныйответна чью-либо просьбу',
 '◆ —\xa0Слышь, Санёк, одолжи пятихатку! — А поебаться тебе не завернуть? Ты мне ещё с прошлого раза штуковину торчишь!',
 'нет, никогда, ни за что, ни в коем случае, ни при каких условиях, разг., нетушки, шиш, фиг, фигу, фигу с дрыгой, фигушки, держи карман шире, ни за какие коврижки, жарг., мол., а банана не хочешь?, а тайгу тебе не пропылесосить?, а спину тебе вареньем не намазать?, а спинку Вам крыжовенным вареньем не намазать?, а в рот тебе не плюнуть жёваной морковкой?, а шнурки тебе не погладить?, а в носе не засвистит?, разг., сниж., в рот тебе ноги у дороги!, эвф., хрена, хренушки, хрен тебе, хрен тебе в зубы, хрен тебе в грызло, хрен тебе на лысый череп, хера, хер тебе, хер тебе в зубы, а хрена с марципанами не хотите?, а попа не слипнется?, а попка не слипнется?, а писюн тебе в фантик не завернуть?, а ху-ху не хо-хо?, эвф., шутл., а липка не спопнется?, жарг., а вафлю в грызло не хочешь?, комп.

In [116]:
a = 'нет, никогда, ни за что, ни в коем случае, ни при каких условиях, разг., лох, дибил'
a.split(', ')
words = ['нет', 'никогда', 'ни за что', 'ни в коем случае', 'ни при каких условиях', 'разг.', 'лох', 'дибил']

# Найдем индекс первого слова с точкой в конце
index = next((i for i, word in enumerate(words) if word.endswith('.')), len(words))

# Получим слова до этого индекса
result = words[:index]

print(result)


['нет', 'никогда', 'ни за что', 'ни в коем случае', 'ни при каких условиях']


In [317]:
import pandas as pd
import re

def clean_text(text):
    if not isinstance(text, str):
        return ''
    return re.sub(r'[◆—/«»]|[А-Яа-я]+\.\)|↑', '', text).strip().replace('\xa0', '')

def generate_replacement_variants(expression, synonyms, original_phrase):
    if any(val is None for val in [expression, synonyms, original_phrase]):
        return []
    
    result_variants = []
    synonyms_parts = [part for part in synonyms.split(', ') if not part.endswith('.')]
    original_phrase_variants = [original_phrase, original_phrase.capitalize()]
    
    for part in synonyms_parts:
        for variant in original_phrase_variants:
            if variant in expression:
                part = part.capitalize()
                new_expression = expression.replace(variant, part.capitalize())
                new_expression = re.sub(r'\s{2,10}', ' ', new_expression)
                result_variants.append(new_expression)
                break

    return result_variants

df['пример_употребления'] = df['пример_употребления'].apply(clean_text)

df['варианты_замены'] = df.apply(
    lambda row: generate_replacement_variants(
        row['пример_употребления'],
        row['синонимы'],
        row['выражение'],
    ),
    axis=1
)
def generate_replacement_variants(expression, synonyms, original_phrase):
    if any(val is None for val in [expression, synonyms, original_phrase]):
        return []
    
    result_parts = []
    synonyms_parts = [part for part in synonyms.split(', ') if not part.endswith('.')]
    original_phrase_variants = [original_phrase, original_phrase.capitalize()]
    
    for part in synonyms_parts:
        for variant in original_phrase_variants:
            if variant in expression:
                result_parts.append(part)
                break

    return result_parts

df['синоним'] = df.apply(
    lambda row: generate_replacement_variants(
        row['пример_употребления'],
        row['синонимы'],
        row['выражение'],
    ),
    axis=1
)


new_rows = []
for row1, row2, row3, row4, row5 in zip(df['выражение'], df['значение'], df['пример_употребления'], df['синоним'], df['варианты_замены']):
    for item1, item2 in zip(row4, row5):
        new_rows.append([row1, row2, row3, item1, item2])

new_df = pd.DataFrame(new_rows, columns=['выражение', 'значение', 'пример_употребления', 'синоним', 'замена'])

new_df

,выражение,значение,пример_употребления,синоним,замена
0,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...",нет,"Слышь, Санёк, одолжи пятихатку! Нет Ты мне ещё..."
1,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...",никогда,"Слышь, Санёк, одолжи пятихатку! Никогда Ты мне..."
2,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...",ни за что,"Слышь, Санёк, одолжи пятихатку! Ни за что Ты м..."
3,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...",ни в коем случае,"Слышь, Санёк, одолжи пятихатку! Ни в коем случ..."
4,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...",ни при каких условиях,"Слышь, Санёк, одолжи пятихатку! Ни при каких у..."
...,...,...,...,...,...
22932,чешется пиздище,"обсц.о женщине, испытывающей желание заняться...","За околицей хуёк, За рекой хуище! Приходи ко...",пизда чешется,"За околицей хуёк, За рекой хуище! Приходи ко м..."
22933,чешется пиздище,"обсц.о женщине, испытывающей желание заняться...","За околицей хуёк, За рекой хуище! Приходи ко...",пизда горит,"За околицей хуёк, За рекой хуище! Приходи ко м..."
22934,шароёбство,обсц.неприкаянность; невостребованность,Унылое шароёбство и вялая самохтонь отныне и н...,шатание,Унылое Шатание и вялая самохтонь отныне и навс...
22935,шароёбство,обсц.неприкаянность; невостребованность,Унылое шароёбство и вялая самохтонь отныне и н...,блуждание,Унылое Блуждание и вялая самохтонь отныне и на...


In [332]:
import re

# Регулярное выражение для поиска только полного совпадения
search_mat = re.compile(r"^(бля|пиздец|хуй|ебать|долбоёб|дохуя|охуенно|поебать|похеру|хули|похуй|нахуй|блять|блядь|еблан|пизда|охуеть)$", re.IGNORECASE)

# Фильтрация DataFrame с использованием регулярного выражения
filtered_df2 = new_df[new_df['выражение'].apply(lambda word: bool(search_mat.fullmatch(word)))]


filtered_df5 = filtered_df2[filtered_df2['синоним'].apply(lambda s: len(s.split()) >= 2)]
filtered_df5

search_mat = re.compile(r"(?iux)(?<![а-яё])(?:(?:(?:у|[нз]а|(?:хитро|не)?вз?[ыьъ]|с[ьъ]|(?:и|ра)[зс]ъ?|(?:о[тб]|п[оа]д)[ьъ]?|(?:\S(?=[а-яё]))+?[оаеи-])-?)?(?:[её](?:б(?!о[рй]|рач)|п[уа](?:ц|тс))|и[пб][ае][тцд][ьъ]).*?|(?:(?:н[иеа]|(?:ра|и)[зс]|[зд]?[ао](?:т|дн[оа])?|с(?:м[еи])?|а[пб]ч|в[ъы]?|пр[еи])-?)?ху(?:[яйиеёю]|л+и(?!ган)).*?|бл(?:[эя]|еа?)(?:[дт][ьъ]?)?|\S*?(?:п(?:[иеё]зд|ид[аое]?р|ед(?:р(?!о)|[аое]р|ик)|охую)|бля(?:[дбц]|тс)|[ое]ху[яйиеё]|хуйн).*?|(?:о[тб]?|про|на|вы)?м(?:анд(?:[ауеыи](?:л(?:и[сзщ])?[ауеиы])?|ой|[ао]в.*?|юк(?:ов|[ауи])?|е[нт]ь|ища)|уд(?:[яаиое].+?|е?н(?:[ьюия]|ей))|[ао]л[ао]ф[ьъ](?:[яиюе]|[еёо]й))|елд[ауые].*?|ля[тд]ь|(?:[нз]а|по)х)(?![а-яё])", re.IGNORECASE)
# Фильтрация таблицы на основе регулярного выражения
filtered_df = filtered_df5[~filtered_df5['синоним'].apply(lambda word: 
                                              bool(search_mat.search(word.lower())))].iloc[:-1]
# Исправленное регулярное выражение
search_mat2 = re.compile(r"палк|гланд|яйц|пис|хер|сос|трах|жоп|сук|манд|ху|фиг|поп|пизд|хрен|рот|залуп|секс|шлюх|↑|бля|ё|еп|мать|еб|хрен|сра|дур|туп|олух|рег|\.|уть|пял|отъ|глуп|мине|чле|ваги|фал|шма|шня|суч|ля|стерв|голый|стоя|конч|\(|\.|рег|курва|ублюд|ок|оч|муд|пох|дау|крет|иди|не|ох|чур|долб|гов|дер|тво|от|въ|бо|вд|голо|дить", re.IGNORECASE)

# Фильтрация DataFrame с использованием регулярного выражения
wiki_df = filtered_df[~filtered_df['синоним'].apply(lambda word: 
                                               bool(search_mat2.search(word)))]
wiki_df = wiki_df[['выражение', 'синоним']]
wiki_df.reset_index(drop=True, inplace=True)
wiki_df

,выражение,синоним
0,долбоёб,христов человек
1,долбоёб,или прикидывается
2,долбоёб,или прикидываешься
3,долбоёб,или прикидывается
4,долбоёб,или прикидываешься
...,...,...
145,хули,с какой целью
146,хули,что толку
147,хули,что тут скажешь
148,хули,одно слово


In [327]:
wiki_df.to_csv('матные_выражения_и_примеры_8.csv')

In [315]:
search_mat = re.compile(r"(?iux)(?<![а-яё])(?:(?:(?:у|[нз]а|(?:хитро|не)?вз?[ыьъ]|с[ьъ]|(?:и|ра)[зс]ъ?|(?:о[тб]|п[оа]д)[ьъ]?|(?:\S(?=[а-яё]))+?[оаеи-])-?)?(?:[её](?:б(?!о[рй]|рач)|п[уа](?:ц|тс))|и[пб][ае][тцд][ьъ]).*?|(?:(?:н[иеа]|(?:ра|и)[зс]|[зд]?[ао](?:т|дн[оа])?|с(?:м[еи])?|а[пб]ч|в[ъы]?|пр[еи])-?)?ху(?:[яйиеёю]|л+и(?!ган)).*?|бл(?:[эя]|еа?)(?:[дт][ьъ]?)?|\S*?(?:п(?:[иеё]зд|ид[аое]?р|ед(?:р(?!о)|[аое]р|ик)|охую)|бля(?:[дбц]|тс)|[ое]ху[яйиеё]|хуйн).*?|(?:о[тб]?|про|на|вы)?м(?:анд(?:[ауеыи](?:л(?:и[сзщ])?[ауеиы])?|ой|[ао]в.*?|юк(?:ов|[ауи])?|е[нт]ь|ища)|уд(?:[яаиое].+?|е?н(?:[ьюия]|ей))|[ао]л[ао]ф[ьъ](?:[яиюе]|[еёо]й))|елд[ауые].*?|ля[тд]ь|(?:[нз]а|по)х)(?![а-яё])", re.IGNORECASE)
# Фильтрация таблицы на основе регулярного выражения
filtered_df = new_df[~new_df['синоним'].apply(lambda word: 
                                              bool(search_mat.search(word.lower())))].iloc[:-1]

# Исправленное регулярное выражение
search_mat2 = re.compile(r"палк|гланд|яйц|пис|хер|сос|трах|жоп|сук|манд|ху|фиг|поп|пизд|хрен|рот|залуп|секс|шлюх|↑|бля|ё|еп|мать|еб|хрен|сра|дур|туп|олух|рег|\.|уть|пял|отъ|глуп|мине|чле|ваги|фал|шма|шня|суч|ля|стерв|голый|стоя|конч|\(|\.|рег|курва|ублюд|ок|оч|муд|пох|дау|крет|иди|не|ох|чур|долб|гов|дер|тво|от|въ|бо|вд|голо|дить", re.IGNORECASE)

# Фильтрация DataFrame с использованием регулярного выражения
filtered_df2 = filtered_df[~filtered_df['синоним'].apply(lambda word: 
                                               bool(search_mat2.search(word)))]

filtered_df2

# Фильтрация DataFrame: разделяем синонимы на слова и исключаем те, которые содержат ровно одно слово
filtered_df3 = filtered_df2[filtered_df2['синоним'].apply(lambda s: len(s.split()) == 1)]

filtered_df3



,выражение,значение,пример_употребления,синоним,замена
1,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...",никогд,"Слышь, Санёк, одолжи пятихатку! Никогда Ты мне..."
6,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...",ши,"Слышь, Санёк, одолжи пятихатку! Шиш Ты мне ещё..."
7,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...",фи,"Слышь, Санёк, одолжи пятихатку! Фиг Ты мне ещё..."
106,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...",таб,"Слышь, Санёк, одолжи пятихатку! Табу Ты мне ещ..."
110,архипиздрит,"обсц.бабник, ловелас, дамский угодник, гуляка","Да, Андрюха тот ещё архипиздрит каждый день н...",бабни,"Да, Андрюха тот ещё Бабник каждый день новая б..."
...,...,...,...,...,...
22921,хуяра,обсц.увелич.кхуй,"Полюбила лейтенанта, А потом полковника! У п...",кута,"Полюбила лейтенанта, А потом полковника! У пол..."
22922,хуяра,обсц.увелич.кхуй,"Полюбила лейтенанта, А потом полковника! У п...",куша,"Полюбила лейтенанта, А потом полковника! У пол..."
22923,хуяра,обсц.увелич.кхуй,"Полюбила лейтенанта, А потом полковника! У п...",салты,"Полюбила лейтенанта, А потом полковника! У пол..."
22934,шароёбство,обсц.неприкаянность; невостребованность,Унылое шароёбство и вялая самохтонь отныне и н...,шатани,Унылое Шатание и вялая самохтонь отныне и навс...


In [204]:
any(len(word) == 1 for word in 'да'.split())

False

In [208]:
a = set(filtered_df3['синоним'].tolist())
print(len(a))

with open('your_file.txt', 'w') as f:
    for line in a:
        f.write(f"{line}\n")

1330


In [171]:
filtered_df2.to_csv('матные_выражения_и_примеры_5.csv')

In [106]:
len(set(filtered_df['выражение'].tolist()))

262

In [32]:
df = df.drop(columns=['синонимы'])
df

,выражение,значение,пример_употребления,варианты_замены
0,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...","Слышь, Санёк, одолжи пятихатку! Нет. Ты мне ещ..."
1,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...","Слышь, Санёк, одолжи пятихатку! Никогда. Ты мн..."
2,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...","Слышь, Санёк, одолжи пятихатку! Ни за что. Ты ..."
3,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...","Слышь, Санёк, одолжи пятихатку! Ни в коем случ..."
4,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...","Слышь, Санёк, одолжи пятихатку! Ни при каких у..."
...,...,...,...,...
22932,чешется пиздище,"обсц.о женщине, испытывающей желание заняться...","За околицей хуёк, // За рекой хуище! // Прих...","За околицей хуёк, // За рекой хуище! // Приход..."
22933,чешется пиздище,"обсц.о женщине, испытывающей желание заняться...","За околицей хуёк, // За рекой хуище! // Прих...","За околицей хуёк, // За рекой хуище! // Приход..."
22934,шароёбство,обсц.неприкаянность; невостребованность,«Унылое шароёбство и вялая самохтонь отныне и ...,«Унылое Шатание. и вялая самохтонь отныне и на...
22935,шароёбство,обсц.неприкаянность; невостребованность,«Унылое шароёбство и вялая самохтонь отныне и ...,«Унылое Блуждание. и вялая самохтонь отныне и ...


In [41]:
df.iloc[:-1].to_csv('матные_выражения_и_примеры2.csv')  

In [40]:
pd.read_csv('матные_выражения_и_примеры2.csv').iloc[:-1]

,Unnamed: 0,выражение,значение,пример_употребления,варианты_замены
0,0,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...","Слышь, Санёк, одолжи пятихатку! Нет. Ты мне ещ..."
1,1,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...","Слышь, Санёк, одолжи пятихатку! Никогда. Ты мн..."
2,2,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...","Слышь, Санёк, одолжи пятихатку! Ни за что. Ты ..."
3,3,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...","Слышь, Санёк, одолжи пятихатку! Ни в коем случ..."
4,4,а поебаться тебе не завернуть?,обсц.отрицательныйответна чью-либо просьбу,"Слышь, Санёк, одолжи пятихатку! А поебаться т...","Слышь, Санёк, одолжи пятихатку! Ни при каких у..."
...,...,...,...,...,...
22931,22931,чешется пиздище,"обсц.о женщине, испытывающей желание заняться...","За околицей хуёк, // За рекой хуище! // Прих...","За околицей хуёк, // За рекой хуище! // Приход..."
22932,22932,чешется пиздище,"обсц.о женщине, испытывающей желание заняться...","За околицей хуёк, // За рекой хуище! // Прих...","За околицей хуёк, // За рекой хуище! // Приход..."
22933,22933,чешется пиздище,"обсц.о женщине, испытывающей желание заняться...","За околицей хуёк, // За рекой хуище! // Прих...","За околицей хуёк, // За рекой хуище! // Приход..."
22934,22934,шароёбство,обсц.неприкаянность; невостребованность,«Унылое шароёбство и вялая самохтонь отныне и ...,«Унылое Шатание. и вялая самохтонь отныне и на...


In [254]:
from datasets import load_dataset
dataset = load_dataset("s-nlp/ru_paradetox")

/Users/anper/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 1116/1116 [00:00<00:00, 180790.36 examples/s]


In [257]:
df_paradetox = dataset['train'].to_pandas()
df_paradetox

,ru_toxic_comment,ru_neutral_comment
0,"и,чё,блядь где этот херой был до этого со свои...","Ну и где этот герой был,со своими доказательст..."
1,"и,чё,блядь где этот херой был до этого со свои...",Где этот герой был до этого со своими доказате...
2,"и,чё,блядь где этот херой был до этого со свои...","и,где этот герой был до этого со своими доказа..."
3,"О, а есть деанон этого петуха?","О, а есть деанон"
4,"херну всякую пишут,из-за этого лайка.долбоебизм.","Чушь всякую пишут, из- за этого лайка."
...,...,...
11085,"Блядь, пусть его уже закроют до конца его дней...","Пусть его уже закроют до конца его дней, он же..."
11086,твоя химия это тотальный пиздец(,твоя химия - это просто кошмар
11087,"меня изнасиловали, мудилка, а тебе пох(((","меня изнасиловали,а тебе всё равно ((("
11088,когда наплоюсь на пидораса и маньяка похож((((...,Когда напьюсь - на маньяка похож...


In [276]:
new_rows = []
for row1, row2 in zip(df_paradetox['ru_toxic_comment'], df_paradetox['ru_neutral_comment']):
        if len(row1.split()) + 2 < len(row2.split()):
            new_rows.append([row1, row2])

ru_toxic = pd.DataFrame(new_rows, columns=['toxic', 'neutral'])

ru_toxic

,toxic,neutral
0,посадить бойцов в кузов им точно пизда,"Посадить бойцов в кузов, и им тогда точно буде..."
1,"потомушто долбоебы, зато щас разнообразного хл...","Потому что не в себе, зато сейчас разнообразно..."
2,за деньги куда хочешь выйдут. дебилы!,За деньги они куда угодно пойдут. Не умные люди.
3,хуйня шо больше показать нечего,Мне не понравилось. Что больше показать нечего ?
4,"вот,вот! почему полицейские не задержали пидор...",Вот вот почему полицейские не задержали челове...
...,...,...
175,пздц штобыломеня обосрали перед всем классом и...,"Кошмар, что было, меня облили грязью перед все..."
176,Данка рассталась со своим( пиздец....родная я ...,Данка рассталась со своим .родная я с тобой .т...
177,Я ДОЛГО НЕ МОГ РЕШИТЬСЯ НА ЭТО.НО ВСЁ ЖЕ ПОСМО...,"Я долго не мог решиться на это, но все же посм..."
178,Что!?!?! СЕКС!!?!?! ГДЕ СЕКС??(ПОЩИМУ Я ПРОПУС...,Что!?!?! Секс!!?!?! Где секс?? ( Почему я проп...


In [284]:
маты = {
    'Охуеть': [
        'Это ошеломляющее событие!',
        'Меня переполняют эмоции!',
        'Я в крайней степени удивлен!',
        'Настолько неожиданно, что у меня нет слов.',
        'Это вызывает у меня чувство крайнего недоумения.',
        'Настолько неординарно, что это меня шокирует.',
        'Мне кажется, это событие достойно моего восхищения.',
        'Я даже не могу поверить в происходящее!',
        'Вызывает крайнее изумление и недоверие.',
        'Такой сюрприз выбивает меня из колеи.'
    ],
    'Ну ты и разъебай': [
        'Ты выделяешься своим экстремальным поведением.',
        'С твоей стороны это было крайне энергично.',
        'Это явно вышло за рамки ожидаемого.',
        'Ты продемонстрировал максимальную степень энтузиазма.',
        'Такой подход трудно не заметить.',
        'Всё, что ты сделал, оставило неизгладимое впечатление.',
        'С твоей стороны это было мощно и напористо.',
        'Ты явно взорвал все вокруг своим поступком.',
        'Это было настолько неожиданно, что просто поражает.',
        'Ты привлекаешь всеобщее внимание своими действиями.'
    ],
    'Как ты меня заебал': [
        'Ты слишком назойлив, я не хочу продолжать разговор.',
        'Твоя настойчивость вызывает у меня раздражение.',
        'Я устал от постоянных повторений с твоей стороны.',
        'Мне сложно находить в себе силы тебя слушать.',
        'Кажется, ты превзошел все допустимые границы.',
        'Ты отнимаешь у меня слишком много энергии.',
        'Мне уже не хочется уделять этому время.',
        'Это начинает действовать мне на нервы.',
        'Ты доводишь меня до предела своим поведением.',
        'Было бы неплохо дать мне передохнуть от твоих слов.'
    ],
    'Ебать': [
        'Меня это сильно удивляет.',
        'Такой поворот событий поражает.',
        'Это вызывает у меня огромное изумление.',
        'Такой неожиданный поворот мне в голову не приходил.',
        'Я поражен в самое сердце этим фактом.',
        'Это вызывает у меня крайнее восхищение.',
        'Я не могу скрыть своего сильного удивления.',
        'Такой сюрприз я не мог предугадать.',
        'Эмоции захлестывают меня при виде этого.',
        'Я поражен до глубины души.'
    ],
    'Нихуя себе тачка': [
        'Какая у тебя крутая машина!',
        'Меня впечатляет твой выбор автомобиля.',
        'Такое авто вызывает у меня восторг.',
        'Это одна из самых привлекательных машин, что я видел.',
        'Автомобиль твоей мечты, без преувеличений.',
        'Видя твой автомобиль, я не могу не восхититься.',
        'Твоя машина выглядит крайне солидно.',
        'Это просто великолепный образец автоискусства.',
        'Вид твоего автомобиля будоражит мое воображение.',
        'Автомобильный шедевр на четырех колесах.'
    ],
    'Ты ахуел': [
        'Своими действиями ты подвергаешь меня изумлению.',
        'Я не ожидала такого вопиющего поведения от тебя.',
        'Твои поступки вызывают у меня крайнее недоумение.',
        'Мне сложно поверить, что ты решил поступить таким образом.',
        'Ты явно перешел все возможные границы.',
        'Мне кажется, твое поведение неуместно.',
        'Я в шоке от твоей наглости.',
        'Это было чересчур вызывающе.',
        'Ты действуешь совершенно непредсказуемо.',
        'Такой поступок требует объяснения.'
    ],
    'Я хуй знает': [
        'Я в крайней степени затруднения ответить.',
        'Хотелось бы мне знать, но увы.',
        'Я тоже не располагаю данной информацией.',
        'Мне сложно дать точный ответ.',
        'Увы, я не компетентен в этом вопросе.',
        'Пока что мне неизвестно, как решить эту задачу.',
        'Я пребываю в состоянии полной неопределенности.',
        'Ответ на этот вопрос мне недоступен.',
        'Надеюсь, вскоре разберусь, но пока не знаю.',
        'Загадка остается для меня неразрешенной.'
    ],
    'Пизда': [
        'Это полный провал.',
        'Была допущена ошибка.',
        'Ужасная ситуация.',
        'Всё идет не по плану.',
        'Такого развития событий мы не ожидали.',
        'Это самое худшее, что могло случиться.',
        'Мы столкнулись с абсолютной катастрофой.',
        'Сложившаяся ситуация просто ужасна.',
        'Всё стало значительно хуже, чем я думал.',
        'Мы попали в крайне неприятное положение.'
    ],
    'Пошел нахуй': [
        'Я с тобой полностью не согласен.',
        'Не отвлекайте меня, пожалуйста.',
        'Мне сложно продолжать с вами разговор.',
        'Давайте прекратим беседу.',
        'Пожалуйста, оставьте меня в покое.',
        'Мне больше не интересно обсуждать это.',
        'Я предпочитаю держаться от тебя подальше.',
        'Не хочу дальше общаться на эту тему.',
        'Твоя настойчивость неуместна.',
        'Мне неприятно продолжать разговор.'
    ],
    'Заебал': [
        'Хватит заставлять себя уговаривать.',
        'Ты слишком назойливый.',
        'Довольно повторять одно и то же.',
        'Мне надоело выслушивать одно и то же.',
        'Ты постоянно отвлекаешь меня.',
        'Мне стало сложно терпеть это.',
        'С тобой сложно общаться.',
        'Ты не оставляешь мне возможности для отдыха.',
        'Пора бы дать мне передышку.',
        'Этот разговор становится слишком долгим.'
    ],
    'Блять': [
        'Ничего себе!',
        'Это было неожиданно.',
        'Меня переполняют эмоции.',
        'Трудно поверить в такое событие.',
        'Сложно выразить мое удивление.',
        'Это превзошло все мои ожидания.',
        'Не могу скрыть своей реакции на это.',
        'Видимо, я оказался совершенно не готов.',
        'Такой поворот мне даже не снился.',
        'Неожиданность зашкаливает!'
    ],
    'Совсем ахуел': [
        'Твое поведение не соответствует моим ожиданиям.',
        'Ты слишком сильно вышел за рамки.',
        'Такое поведение вызывает крайнее недоумение.',
        'Мне сложно поверить в твою наглость.',
        'Это абсолютно неприемлемо.',
        'Ты явно перешел все возможные границы.',
        'Такого я не ожидал даже от тебя.',
        'Твои действия слишком вызывающие.',
        'Мне кажется, ты совсем перестал себя контролировать.',
        'Такое поведение меня огорчает.'
    ],
    'Тебе въебать?': 'Ты хочешь отстоять свою правоту в поединке со мной?'
}
новые_маты = {
    'Ну просто охереть': [
        'Это событие полностью выбивает меня из колеи.',
        'Я поражен и не могу подобрать слова.',
        'Такой исход меня по-настоящему удивляет.',
        'Это вызывает у меня крайнее изумление.',
        'Мне сложно выразить словами свое удивление.',
        'Это происходит вопреки всем моим ожиданиям.',
        'Я не могу поверить в то, что вижу.',
        'Это одно из самых неожиданных событий в моей жизни.',
        'Настолько удивительно, что у меня просто нет слов.',
        'Мое восхищение и удивление зашкаливают.'
    ],
    'Охуеть можно': [
        'Меня сильно удивляет такое нерациональное поведение.',
        'Трудно представить, что кто-то мог так поступить.',
        'Это кажется мне крайне неосмысленным.',
        'Я не ожидал такой степени необдуманности.',
        'Такое действие выходит за рамки разумного.',
        'Не могу поверить в такую ошибку.',
        'Мне трудно осознать, что это произошло.',
        'Это вызывает у меня глубокое недоумение.',
        'Трудно выразить, насколько это необъяснимо.',
        'Меня поражает отсутствие логики в этом.'
    ],
    'Охуеть': [
        'Настолько неожиданный поворот событий меня шокирует.',
        'Это в корне меняет мое восприятие ситуации.',
        'Я и представить не мог, что все изменится так резко.',
        'Мир будто изменился у меня на глазах.',
        'Такое изменение вызывает у меня сильное изумление.',
        'Это заставляет меня пересмотреть мои взгляды.',
        'Моя картина мира претерпела серьезные изменения.',
        'Такое событие способно изменить восприятие всего.',
        'Я поражен масштабами этих перемен.',
        'Все происходит совершенно неожиданно.',
        'Мне кажется, это не такая уж важная информация.',
        'Это вызывает у меня сомнение в значимости сообщения.',
        'Мне трудно воспринять это как что-то значимое.',
        'Это кажется мне малозначительным.',
        'Такое событие не стоит того, чтобы на него обращали внимание.',
        'Я не считаю эту новость действительно стоящей.',
        'Мне трудно поверить, что это кого-то может заинтересовать.',
        'Возможно, это не самое важное событие.',
        'Это выглядит как малозначительное событие.',
        'Информация не производит большого впечатления.',
        'Такой уровень креативности я даже не ожидал.',
        'Невероятно, что кто-то мог придумать это.',
        'Это требует особого мышления, чтобы додуматься до такого.',
        'Трудно представить, как это вообще пришло в голову.',
        'Идея явно выходит за рамки обыденного.',
        'Это вызывает у меня сильное удивление своей оригинальностью.',
        'Кто бы мог подумать, что можно придумать такое.',
        'Это весьма неординарное мышление.',
        'Идея поражает своей нестандартностью.',
        'Это действительно требует фантазии и креатива.',
        'Я в крайней степени поражен и не знаю, что сказать.',
        'Мое изумление не знает границ.',
        'Меня это событие полностью ошарашило.',
        'Я сижу в полном недоумении.',
        'Такое впечатление, что я нахожусь в шоке.',
        'Это вызывает у меня сильное потрясение.',
        'Событие повергло меня в состояние замешательства.',
        'Я чувствую себя совершенно потерянным.',
        'Это настолько неожиданно, что я не могу среагировать.',
        'Мое удивление зашкаливает.',
        'Такой исход вызывает у меня недоумение.',
        'Мне интересно, что последует за этим.',
        'Это событие оставляет меня в раздумьях.',
        'Я не знаю, что ожидать дальше.',
        'Ситуация остается непредсказуемой.',
        'Я с любопытством жду дальнейшего развития событий.',
        'Это вызывает у меня чувство ожидания.',
        'Интересно, как это повлияет на будущее.',
        'Такой поворот событий интригует меня.',
        'Я надеюсь узнать, что будет дальше.'
    ],
    'Какая-то хуйня': [
        'Мне кажется, это событие не заслуживает особого внимания.',
        'Я не нахожу в этом ничего примечательного.',
        'Это не вызывает у меня сильных эмоций.',
        'Событие кажется мне вполне обычным.',
        'Меня это совершенно не впечатляет.',
        'В этом нет ничего, что могло бы меня заинтересовать.',
        'Это кажется мне весьма посредственным.',
        'Я не нахожу в этом ничего особенного.',
        'Событие не вызывает во мне восхищения.',
        'Мое восприятие этого события остается нейтральным.'
    ],
    'Пиздец': [
        'Сложно поверить, что это действительно происходит.',
        'Я нахожусь в полном изумлении от происходящего.',
        'Это событие кажется мне совершенно нереальным.',
        'Я поражен тем, что это происходит на моих глазах.',
        'Мне трудно осознать, что это не сон.',
        'Это вызывает у меня сильное чувство недоверия.',
        'Ситуация кажется абсолютно невероятной.',
        'Это удивляет меня больше, чем я могу выразить.',
        'Мое сознание не успевает за реальностью.',
        'Такое событие меня просто ошеломляет.'
    ],
    'Ты ебанулся?': [
        'Трудно представить, как ты это придумал.',
        'Меня удивляет твое креативное мышление.',
        'Это поистине оригинальная идея.',
        'Мне интересно, что побудило тебя к этому.',
        'Ты явно мыслитель не от мира сего.',
        'Такой замысел вызывает у меня любопытство.',
        'Твоя фантазия поражает меня.',
        'Мне бы хотелось понять, что тобой двигало.',
        'Эта мысль кажется мне совершенно необычной.',
        'Ты придумал нечто поистине нестандартное.'
    ]
}

дополнительные_маты = {
    'Пиздец': [
        'Это приводит меня в крайнее замешательство.',
        'Ситуация достигла своего апогея в плане сложности.',
        'Мне сложно описать, насколько это серьезно.',
        'Это настоящий провал, который невозможно игнорировать.',
        'Ситуация выглядит чрезвычайно неблагоприятно.',
        'Это событие вызывает у меня крайнюю обеспокоенность.',
        'Все пошло совсем не так, как планировалось.',
        'Я даже не мог представить такой плачевный исход.',
        'Это действительно ужасный поворот событий.',
        'Такое развитие событий поражает своей критичностью.',
        'Это разрушает все ранее выстроенные планы.',
        'Такой исход приводит к полной дезорганизации.',
        'Я не могу поверить, что все так обрушилось.',
        'Это событие нарушает всю систему.',
        'Масштаб ситуации кажется мне грандиозным.',
        'Все правила, казалось бы, потеряли силу.',
        'Это полностью меняет всю картину происходящего.',
        'Мое представление о порядке вещей разрушено.',
        'Такое событие выбивает из колеи.',
        'Это явное нарушение всех возможных ожиданий.',
        'Меня оставляет безмолвным происходящее.',
        'Я не могу подобрать слов, чтобы выразить свое удивление.',
        'Ситуация оставляет меня в полном недоумении.',
        'Я совершенно не знаю, что сказать об этом.',
        'Мое удивление не поддается описанию.',
        'Я нахожусь в глубоком молчании от произошедшего.',
        'Все это кажется мне крайне поразительным.',
        'Мне трудно выразить свое удивление словами.',
        'Я в замешательстве и не нахожу слов.',
        'Такое развитие событий лишает меня речи.'
    ],
    'Какого хуя?': [
        'Меня искренне удивляет, что это имеет место быть.',
        'Мне трудно понять, почему это происходит.',
        'Это событие вызывает у меня сильное недоумение.',
        'Я не ожидал такого развития событий.',
        'Это совершенно не укладывается в моем сознании.',
        'Мне непонятно, как вообще такое возможно.',
        'Это вызывает у меня чувство крайнего удивления.',
        'Почему это происходит, остается для меня загадкой.',
        'Такой поворот событий совершенно не вписывается в мою логику.',
        'Меня поражает нелогичность происходящего.'
    ],
    
    'Ты меня конкретно заебал уже': [
        'Твоя настойчивость вызывает у меня чувство усталости.',
        'Мне сложно продолжать с тобой общение в таком формате.',
        'Я ощущаю себя уставшим от твоих действий.',
        'Мне кажется, ты переусердствовал с настойчивостью.',
        'Я нуждаюсь в перерыве от нашего взаимодействия.',
        'Сложно не заметить, насколько ты меня утомил.',
        'Ты слишком часто повторяешь одно и то же.',
        'Мне нужно немного пространства от твоих идей.',
        'Я чувствую сильное напряжение в разговоре с тобой.',
        'Такое общение вызывает у меня чувство раздражения.'
    ],
    'Что за херня?': [
        'Мне трудно понять суть происходящего.',
        'Это вызывает у меня чувство сильного замешательства.',
        'Ситуация кажется крайне странной.',
        'Мне непонятно, как это могло произойти.',
        'Я совершенно сбит с толку тем, что вижу.',
        'Это событие кажется мне абсолютно нелогичным.',
        'Я не могу найти объяснение происходящему.',
        'Все выглядит абсолютно непредсказуемо.',
        'Сложно поверить в то, что это реально.',
        'Я пребываю в состоянии полного непонимания.'
    ],
    'Пиздец полный, просто охуеть': [
        'Это ситуация вызывает у меня крайнюю обеспокоенность.',
        'Я в глубоком замешательстве от происходящего.',
        'Все складывается самым неблагоприятным образом.',
        'Мне сложно поверить в такую критическую ситуацию.',
        'Это крайне неблагоприятный поворот событий.',
        'Такое развитие событий кажется мне катастрофическим.',
        'Я нахожусь в полном недоумении.',
        'Сложно описать всю серьезность произошедшего.',
        'Моя реакция полна глубокого изумления.',
        'Это вызывает у меня сильное чувство беспокойства.'
    ],
    'Охуеть': [
        'Это событие вызывает у меня максимальное удивление.',
        'Я чувствую, что скоро буду в полном шоке.',
        'Мое изумление достигнет предела.',
        'Такое развитие событий сильно меня поражает.',
        'Я не могу поверить в происходящее.',
        'Это вызывает у меня невероятное волнение.',
        'Мне сложно представить себе такой поворот.',
        'Я в ожидании чего-то совершенно невероятного.',
        'Это обещает быть чем-то совершенно непредсказуемым.',
        'Мое предвкушение шока зашкаливает.',
        'Твое заявление меня сильно удивило.',
        'Я не ожидал услышать от тебя что-то подобное.',
        'Это звучит крайне необычно.',
        'Мне сложно поверить в сказанное тобой.',
        'Ты действительно превзошел все ожидания.',
        'Это было очень неожиданно.',
        'Ты сделал невероятно удивительное замечание.',
        'Я поражен твоей оригинальностью.',
        'Это заявление стало настоящим сюрпризом.',
        'Ты вызвал у меня огромное изумление.'
    ],
    'Что ты пизданул?': [
        'Ты осознаешь, что сказал что-то необоснованное?',
        'Мне кажется, твои слова не имеют достаточной аргументации.',
        'Ты точно уверен в том, что это логично?',
        'Мне трудно поверить в разумность твоих слов.',
        'Ты уверен, что то, что ты сказал, имеет смысл?',
        'Мне кажется, в твоем высказывании мало логики.',
        'Ты продумал, что сказал, или это спонтанно?',
        'Уверен ли ты в своей точке зрения?',
        'Это звучит как что-то очень противоречивое.',
        'Мне сложно воспринять это всерьез.',
        'Ты действительно озвучил это вслух?',
        'Я не ожидал услышать такие слова.',
        'Это вызывает у меня большое удивление.',
        'Мне трудно поверить, что это прозвучало от тебя.',
        'Ты осознаешь, что только что сказал?',
        'Это крайне неожиданное заявление с твоей стороны.',
        'Меня это сильно поразило.',
        'Ты действительно это высказал?',
        'Мне сложно представить, что это реально.',
        'Это произвело на меня сильное впечатление.'
    ]
}


a = {}
a.update(маты)
a.update(новые_маты)
a.update(дополнительные_маты)

In [288]:
len(a.keys()), len(маты.keys()) + len(новые_маты.keys()) + len(дополнительные_маты.keys())

(23, 26)

In [339]:
obscenes = {'toxic': [], 'neutral': []}

for key, value in a.items():
    if isinstance(value, list):
        for v in value:
            obscenes['toxic'].append(key.lower())
            obscenes['neutral'].append(v.lower()[:-1])
    else:
        obscenes['toxic'].append(key.lower())
        obscenes['neutral'].append(v.lower()[:-1])

obscenes_df = pd.DataFrame.from_dict(obscenes)
obscenes_df

,toxic,neutral
0,охуеть,это событие вызывает у меня максимальное удивл...
1,охуеть,"я чувствую, что скоро буду в полном шоке"
2,охуеть,мое изумление достигнет предела
3,охуеть,такое развитие событий сильно меня поражает
4,охуеть,я не могу поверить в происходящее
...,...,...
256,что ты пизданул?,это крайне неожиданное заявление с твоей стороны
257,что ты пизданул?,меня это сильно поразило
258,что ты пизданул?,ты действительно это высказал
259,что ты пизданул?,"мне сложно представить, что это реально"


In [341]:
obscenes_df

,toxic,neutral
0,охуеть,это событие вызывает у меня максимальное удивл...
1,охуеть,"я чувствую, что скоро буду в полном шоке"
2,охуеть,мое изумление достигнет предела
3,охуеть,такое развитие событий сильно меня поражает
4,охуеть,я не могу поверить в происходящее
...,...,...
256,что ты пизданул?,это крайне неожиданное заявление с твоей стороны
257,что ты пизданул?,меня это сильно поразило
258,что ты пизданул?,ты действительно это высказал
259,что ты пизданул?,"мне сложно представить, что это реально"


In [342]:
wiki_df = wiki_df.rename(columns={'выражение': 'toxic', 'синоним': 'neutral'})
concat = pd.concat([wiki_df, obscenes_df])
concat.reset_index(drop=True, inplace=True)
concat

,toxic,neutral
0,долбоёб,христов человек
1,долбоёб,или прикидывается
2,долбоёб,или прикидываешься
3,долбоёб,или прикидывается
4,долбоёб,или прикидываешься
...,...,...
406,что ты пизданул?,это крайне неожиданное заявление с твоей стороны
407,что ты пизданул?,меня это сильно поразило
408,что ты пизданул?,ты действительно это высказал
409,что ты пизданул?,"мне сложно представить, что это реально"


In [343]:
concat.to_csv('toxic_short.csv')

In [338]:
ru_toxic.to_csv('toxic_sentences.csv')